<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-необходимых-библиотек" data-toc-modified-id="Импорт-необходимых-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт необходимых библиотек</a></span></li><li><span><a href="#Объявление-функций" data-toc-modified-id="Объявление-функций-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Объявление функций</a></span><ul class="toc-item"><li><span><a href="#Загрузка-датасета" data-toc-modified-id="Загрузка-датасета-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Загрузка датасета</a></span></li><li><span><a href="#Проверка-загрузки" data-toc-modified-id="Проверка-загрузки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Проверка загрузки</a></span></li><li><span><a href="#Распаковка-списка-с-жанрами-в-строку" data-toc-modified-id="Распаковка-списка-с-жанрами-в-строку-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Распаковка списка с жанрами в строку</a></span></li><li><span><a href="#Парсинг-текста" data-toc-modified-id="Парсинг-текста-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Парсинг текста</a></span></li></ul></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Парсинг-отсутствующих-текстов" data-toc-modified-id="Парсинг-отсутствующих-текстов-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Парсинг отсутствующих текстов</a></span></li></ul></div>

# Парсинг данных

## Импорт необходимых библиотек

In [1]:
# импорты из стандартной библиотеки
import os
import re
import warnings

# импорты сторонних библиотек
import numpy as np
import pandas as pd
import pickle
import joblib
import lyricsgenius
from pandarallel import pandarallel
from tqdm import tqdm

# настройки
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("max_colwidth", 100)

# константы
DATA_PATH = "../data/raw_data/"
PARSED_DATA_PATH = "../data/parsed_data/"


## Объявление функций

### Загрузка датасета

In [2]:
def get_first_dataset(work_dir, meta_json, lyrics_json, covers_json):
    """
    Функция загрузки первичного датасета.

    Параметры:

     work_dir: папка с файлами json, в которых содержатся данные.
     meta_json, lyrics_json, covers_json: файлы с данными

    Возвращаемое значение:

     data: объединенный датасет из файлов meta_json, lyrics_json, covers_json по ключу 'track_id'

    """
    # загрузка метаданных
    meta = pd.read_json(
        os.path.join(work_dir, meta_json), orient="columns", lines=True
    )

    # загрузка имеющихся в наличии текстов песен
    lyrics = pd.read_json(
        os.path.join(work_dir, lyrics_json), orient="columns", lines=True
    )

    # загрузка разметки
    covers = pd.read_json(
        os.path.join(work_dir, covers_json), orient="columns", lines=True
    )

    # вызов функции unpack_list для распаковки жанров
    tqdm.pandas(desc="Распаковка списков в столбце genres.")
    meta["genres"] = meta["genres"].progress_apply(unpack_list)

    # объединение covers и meta
    data = pd.merge(covers, meta, how="left", on=["track_id"])

    # объединение data и lyrics
    data = pd.merge(
        data, lyrics[["text", "track_id"]], how="left", on=["track_id"]
    )

    print(f"Датасет сформирован")

    return data

### Проверка загрузки

In [3]:
def display_dataset_info(dataset):
    """
    Функция проверки загрузки: вывод первых 5 строк, информации о датафрейме и наличии дубликатов.

    Параметры:

         dataset: исследуемый датасет.

    Возвращаемое значение:

         Первые 5 строк датафрейма, метод .info() и метод .describe(include="all")
    """
    print("-" * 35 + "SAMPLE" + "-" * 35)
    display(dataset.head())
    print()
    print("-" * 36 + "INFO" + "-" * 36)
    print()
    display(dataset.info())
    print("-" * 34 + "DESCRIBE" + "-" * 34)
    display(dataset.describe(include="all"))

### Распаковка списка с жанрами в строку

In [4]:
def unpack_list(row):
    """
    Функция для распаковки значений списка в столбце genres в датасете meta.

    Параметры:

         row: жанры представленны в виде списка, например ["POP", "ELECTRONIC"]

    Возвращаемое значение:

         - "UNKNOWN", если в данных пропуск или пустой список;
         - row[0], если в списке только один элемент;
         - ", ".join(row), если в списке 2 и более элементов.

    """
    # проверка на пропуск
    if row is None:
        return "UNKNOWN"

    # проверка на пустой список
    elif len(row) == 0:
        return "UNKNOWN"

    # проверка на 1 элемент
    elif len(row) == 1:
        return row[0]

    # проверка на 2 и более элементов в списке
    else:
        return ", ".join(row)

### Парсинг текста

In [5]:
def parse_text(data):
    """
    Функция для парсинга отсутсвующих текстов.

    Параметры:

         data: исходный датасет

    Возвращаемое значение:

         - lyrics, если он есть в базе genius.com;
         - 'No text', если текста нет в базе genius.com;
         - data["text"], если текст есть в датасете.
    """
    # проверка на пропуск текста
    if pd.isnull(data["text"]):
        try:
            # запрос к genius.com по названию трека
            song = genius.search_song(data["title"], get_full_info=False)

            # провека есть ли текст в базе сайта и запись текста в датасет
            if song:
                lyrics = song.lyrics
                return lyrics

            # если текста в базе сайта нет возврат "No text"
            else:
                return "No text"

        # возможна ошибка на самом сайте, неправильное формирование поиска сайтом - возврат "No text"
        except:
            print("Ошибка сайта, пропускаем трек")
            return "No text"

    # если текст есть в датасете возвращаем текст из датасета
    return data["text"]

## Загрузка данных

In [6]:
data = get_first_dataset(DATA_PATH, "meta.json", "lyrics.json", "covers.json")


Распаковка списков в столбце genres.: 100%|██████████████████████████████████| 71769/71769 [00:00<00:00, 596235.86it/s]


Датасет сформирован


## Парсинг отсутствующих текстов

Парсинг будет осуществлен с сайта [genius.com](https://genius.com/) при помощи *genius API*.

Для этого потребовалось:

1. Создать аккаунт на сайте [genius.com](https://genius.com/)
2. Посетить страницу разработчика [Genius](https://genius.com/api-clients)
3. Создать клиента API
4. Заполнить форму для создания клиента API
5. Сохранить клиента API
6. В результате получить страницу клиента API, где будет отображаться токен доступа (Client Access Token).

In [7]:
# создание подключения с использованием созданного Client Access Token
genius = lyricsgenius.Genius(
    "PNUx_CdY0pyHpnWJL6QAyp6vdB3tsQYLlXRHX_tR8YvC49rKr2w3OepkaF7oscT-",
    timeout=600,
)


<div style="border:solid green 2px; padding: 20px">
    
Процесс парсинга занимает 12-15 часов!!!

**Парсинга текста**

In [8]:
pandarallel.initialize()
data["text"] = data.parallel_apply(parse_text, axis=1)

**Сохранение датасета**

In [9]:
data.to_pickle(PARSED_DATA_PATH + "parsed_data", compression="zip")
